## Estimating average 2D localization precision from experimental data

This notebook explains an approach to estimate 2D localization fitting performance that can be used on experimental data. The approach is based on histogramming nearest neighbor distances in adjacent frames. In this example we're going to use `3D-DAOSTORM` algorithm to do the finding/fitting for simplicity, but this approach can be used with any finding/fitting algorithm.

Reference:
* [Endesfelder et al, Histochemistry and Cell Biology, 2014](https://doi.org/10.1007/s00418-014-1192-3).


### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [7]:
import matplotlib
import matplotlib.pyplot as pyplot
import numpy
import os
import scipy
import scipy.optimize


numpy.random.seed(1)

### Fitting Functions

These are what we'll use to fit for the average localization performance.

In [3]:
import storm_analysis.sa_library.ia_utilities_c as iaUtilsC
import storm_analysis.sa_library.sa_h5py as saH5Py

def calcPOfD(hdf5_name, bin_size = 5.0, max_cnts = 10000, n_bins = 40, pixel_size = 100.0, start_frame = 0):
    """
    Calculate the p(d) histogram.
    
    The default is nearest neighbor distances out to 200nm with a 5nm bin
    size and total of 10000 counts (more or less) in the histogram.
    
    hdf5_name - The name of the HDF5 localizations file.
    bin_size - Bin size in nanometers.
    max_cnts - (Approximate) maximum number of events in the histogram
    n_bins - The number of bins in the histogram.
    pixel_size - The size of a pixel in nanometers.
    start_frame - The first frame to include in the analysis.
    """
    
    p_x = numpy.arange(n_bins) * bin_size + 0.5 * bin_size
    p_d = numpy.zeros(n_bins)

    cnts = 0
    l_x = None
    l_y = None
    last_fnum = -10
    with saH5Py.SAH5Py(hdf5_name) as h5:
        for fnum, locs in h5.localizationsIterator(fields = ["x", "y"]):
        
            # Skip first N frames:
            if (fnum < start_frame):
                continue
        
            # Check for empty frame
            if bool(locs):
            
                # Check for sequential frames.
                if (fnum == (last_fnum + 1)):
                
                    # Find nearest localizations in previous frame to 
                    # localizations in current frame.
                    dist = iaUtilsC.peakToPeakDistAndIndex(locs["x"], locs["y"], l_x, l_y)[0]
                
                    # Convert distance in pixels to nanometers.
                    dist = dist * pixel_size
                
                    # Convert to bin size.
                    dist = dist/bin_size
                
                    for i in range(dist.size):
                        index = int(round(dist[i]))
                        if (index >= 0) and (index < p_d.size):
                            p_d[index] += 1
                            cnts += 1
                        
                    if (cnts > max_cnts):
                        break
                
                l_x = locs["x"]
                l_y = locs["y"]
            
            last_fnum = fnum
            
    return [p_x, p_d]

# For single term p(d) function to data.
def fitPOfDDist(x, y, p0):
    [coeff, var_matrix] = scipy.optimize.curve_fit(pOfDDist, x, y, p0=p0)
    return [coeff, pOfDDist(x, *coeff)]

# Fit corrected p(d) function to data.
def fitPOfDDistCor(x, y, p0):
    [coeff, var_matrix] = scipy.optimize.curve_fit(pOfDDistCor, x, y, p0=p0)
    return [coeff, pOfDDistCor(x, *coeff)]

# Single term p(d) function
def pOfDDist(x, *p):
    A1, sig_smlm = p

    t1 = A1 * x/(2 * sig_smlm * sig_smlm) * numpy.exp(-x*x/(4.0 * sig_smlm * sig_smlm))
    return t1

# Corrected p(d) function
def pOfDDistCor(x, *p):
    A1, A2, A3, sig_smlm, w, dc = p
    dd = x - dc

    t1 = A1 * x/(2 * sig_smlm * sig_smlm) * numpy.exp(-x*x/(4.0 * sig_smlm * sig_smlm))
    t2 = A2 * 1.0/numpy.sqrt(2.0 * numpy.pi * w * w) * numpy.exp(-dd*dd/(2.0*w*w))
    t3 = A3 * x
    return t1 + t2 + t3

# Pretty print the corrected p(d) coefficients.
def prettyPrintCoeff(coeff):
    print("     A1: {0:.1f}".format(coeff[0]))
    print("     A2: {0:.1f}".format(coeff[1]))    
    print("     A3: {0:.2f}".format(coeff[2]))
    print("  sigma: {0:.2f}".format(coeff[3]))
    print("      w: {0:.2f}".format(coeff[4]))
    print("     dc: {0:.2f}".format(coeff[5]))
    

### Localizations on a grid example

Create a movie of localizations on a grid.

In [ ]:
import storm_analysis.jupyter_examples.est_fit_prec as estFitPrec

# Set background, signal
estFitPrec.bg = 20
estFitPrec.signal = 2000

# Make parameters file for analysis
estFitPrec.createParametersFile()

# Create ground truth localizations file
estFitPrec.createLocalizationsGrid()

# Create movie
estFitPrec.createMovieGrid(50)


Analyze the movie.

In [ ]:
import storm_analysis.daostorm_3d.mufit_analysis as mfit

# Remove stale results, if any.
if os.path.exists("grid_test.hdf5"):
    os.remove("grid_test.hdf5")
    
# (Re)run the analysis.
mfit.analyze("grid.tif", "grid_test.hdf5", "dao3d_analysis.xml")

Calculate p(d) histogram.

In [ ]:
[p_x, p_d] = calcPOfD("grid_test.hdf5", bin_size = 2.0, pixel_size = estFitPrec.pixel_size)

Fit uncorrected p(d) and compare to the error estimated from the ground truth locations and to the Cramer-Rao estimate.

In this example this is all that is necessary as all the localizations were on and should be detected in every frame.

In [ ]:
import storm_analysis.sa_utilities.finding_fitting_error as ffe
import storm_analysis.sa_utilities.mortensen as mortensen

[coeff, fit] = fitPOfDDist(p_x, p_d, [numpy.max(p_d), 5.0])

pyplot.bar(p_x, p_d, 1.5)
pyplot.plot(p_x, fit, color = "black")
pyplot.show()

# Ground truth comparison estimate.
[dx, dy, dz] = ffe.findingFittingErrorHDF5File("grid_ref.hdf5", "grid_test.hdf5")
gt_sigma = 0.5*(numpy.std(dx) + numpy.std(dy))

# Cramer-Rao bound.
cr_sigma = mortensen.cramerRaoBound(estFitPrec.signal, 
                                    estFitPrec.bg, 
                                    estFitPrec.pixel_size, 
                                    estFitPrec.pixel_size * 1.5)

print("Average precision estimate: {0:.2f}nm".format(coeff[1]))
print("Ground-truth comparison: {0:.2f}nm".format(gt_sigma))
print("Cramer-Rao bound: {0:.2f}nm".format(cr_sigma))

### Randomly distributed localizations



In [ ]:
# Create ground truth localizations file
estFitPrec.createLocalizationsRandom()

# Create movie, 250 frames is enough to reach the default max_cnts argument for calcPOfD().
estFitPrec.createMovieRandom(250)

Analyze the movie.

In [ ]:
# Remove stale results, if any.
if os.path.exists("random_test.hdf5"):
    os.remove("random_test.hdf5")
    
# (Re)run the analysis.
mfit.analyze("random.tif", "random_test.hdf5", "dao3d_analysis.xml")

Calculate p(d) histogram.

In [ ]:
[p_x, p_d] = calcPOfD("random_test.hdf5", bin_size = 2.0, pixel_size = estFitPrec.pixel_size)

Fit uncorrected p(d) to get initial estimates for some of the terms. 

Note that this deviates significantly from the actual distribution.

In [ ]:
[coeff, fit] = fitPOfDDist(p_x, p_d, [numpy.max(p_d), 5.0])

pyplot.bar(p_x, p_d, 1.5)
pyplot.plot(p_x, fit, color = "black")
pyplot.show()

print("Average precision estimate: {0:.2f}nm".format(coeff[1]))
print("Cramer-Rao bound: {0:.2f}nm".format(cr_sigma))

Fit corrected p(d).

This should match the actual distribution much better. Note that this fit is not perfectly stable so in practice you may have to adjust the starting parameters.

The localization precision is now significantly worse than the Cramer-Rao bound. There are several reasons for this including overlapping localizations and localizations that are only on for part of frame. In the simulation the localization on times are exponentially distributed with an average on time of a single frame.

In [ ]:
x_o = [coeff[0], 0.1 * coeff[0], 0.0, coeff[1], coeff[1], 3 * coeff[1]]
print("Initial guess:")
prettyPrintCoeff(x_o)
print()

[coeff_cor, fit_cor] = fitPOfDDistCor(p_x, p_d, x_o)
print("After fitting:")
prettyPrintCoeff(coeff_cor)
print()

pyplot.bar(p_x, p_d, 1.5)
pyplot.plot(p_x, fit_cor, color = "black")
pyplot.show()

print("Corrected average precision estimate: {0:.2f}nm".format(coeff_cor[3]))
print("Cramer-Rao bound: {0:.2f}nm".format(cr_sigma))

# Part 3: General questions

**4.  List 4 or more differences between the images you created and the neuronal data analyzed in part 1. Comment on the size, shapes of the PSF, differences in how dense the spacing is between spots, etc.**


**5. Our ability to resolve structures in an image not only requires many localizations at high precision but also depends on the density of measured emitters.**


**a. Explain why density would matter here. Please relate your answer to Nyquist sampling.**

As per the Nyquist sampling theory, we require approximately two data points per resolution unit. Thus, for a two-dimensional image having spatial features of size $\alpha$, the minimum required molecular density of localized  probes necessary to meet the Nyquist criterion is
\begin{equation}
\text{Nyquist Molecular Density} \approx \Big(\frac{2}{\alpha}\Big)^2
\end{equation}

**b. Sketch a sample/describe an experiment that would allow you to determine the resolution of your reconstruction. Give a short description of your illustration.**


 The Abbe limit of resolution for optical images is approximately 0.22 micrometers (using visible light), meaning that a digitizer must be capable of sampling at intervals that correspond in the specimen space to 0.11 micrometers or less. A digitizer that samples the specimen at 512 pixels per horizontal scan line would have to produce a maximum horizontal field of view of 56 micrometers (512 x 0.11 micrometers) in order to conform to the Nyquist criterion. An interval of 2.5 to 3 samples for the smallest resolvable feature is suggested to ensure adequate sampling for high resolution imaging. ([Reference here](https://zeiss-campus.magnet.fsu.edu/articles/basics/digitalimaging.html))



**6. Would super-resolution imaging would be useful for measuring the cellular component you presented in class. Explain your answer (3-6 sentences).**

Specific tools to visualize ECM components are essential. With optical microscopy, it is possible to better understand the spatial, temporal, and molecular changes that occur in the ECM, as well as cell adhesion points, cytoskeleton morphology, and other features of such a complex microenvironment. Recent developments and applications of super resolution imaging have been used to address increasingly complex biological questions related to ECM remodeling.

